In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()


(6759, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,7.209309,1.860841,2.083642,0.111400,0.111400,1.8857,0.111400
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,7.0,1.9587,0.0824,6.641782,1.814416,2.009966,0.097775,0.097775,1.9587,0.082400
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,6.989511,1.796061,2.104793,0.154366,0.154366,1.7824,0.154366
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,7.0,1.7963,0.0565,6.618147,1.799499,1.991288,0.095895,0.095895,1.7963,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,6.939422,1.859515,2.032438,0.086461,0.086461,2.0053,0.086461


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.111400,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9587,0.082400,7.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.154366,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.7963,0.067659,7.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.086461,2.0


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.7963  0.111400  0.082400   
2              Runa Fremstad  ...   1.9581  1.9854  0.086461  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9113  0.067659  0.100600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.102900  0.094482   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.154366  0.067659      1.0  7.0  1.0  7.0  
2        0.067659  0.086115      2.0  5.0  5.0  3.0  
3        0.141629  0.067659      4.0  7.0  1.0  8.0  
4        0.135658  0.108649      4.0  0.0  1.0  2.0  
5        0.095300  0.128324      7.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,31.764337,69.885701,108.031336,154.414716,49.035446,91.828721,142.393593,190.951232,39.564176,80.274245,...,32.132384,41.737356,49.035446,47.505434,58.328542,54.536690,39.564176,40.512037,43.358461,47.657695
2,37.614954,77.331738,126.928165,181.483027,53.022743,98.456745,152.218743,213.973430,44.637212,87.292951,...,45.385207,48.617899,53.022743,50.552995,59.019893,68.279975,44.637212,42.488228,51.724696,57.515781
3,34.654088,70.679569,112.239959,163.986758,45.288614,89.536932,144.387197,199.221325,39.719698,79.475188,...,35.841118,46.864176,45.288614,48.267807,62.188892,60.822314,39.719698,39.661056,47.237352,53.473302
4,34.732924,69.143257,110.078726,161.701158,53.819722,105.041001,153.325296,210.649615,43.203166,84.466721,...,33.982877,43.879477,53.819722,58.770173,58.228965,67.238285,43.203166,40.925208,44.530412,54.248889
5,32.383791,67.606076,112.946610,160.381883,48.525258,88.692811,141.339266,198.223496,39.676606,77.440974,...,40.461360,40.166567,48.525258,45.211190,58.836387,66.252705,39.676606,37.621297,48.769067,51.394929


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.11,0.08,0.15,0.07,1.0,7.0,1.0,7.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.09,0.09,0.07,0.09,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.91,0.07,0.10,0.14,0.07,4.0,7.0,1.0,8.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.14,0.11,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.10,0.09,0.10,0.13,7.0,2.0,6.0,3.0


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,32.132384,41.737356,49.035446,47.505434,58.328542,54.536690,39.564176,40.512037,43.358461,47.657695
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.385207,48.617899,53.022743,50.552995,59.019893,68.279975,44.637212,42.488228,51.724696,57.515781
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,35.841118,46.864176,45.288614,48.267807,62.188892,60.822314,39.719698,39.661056,47.237352,53.473302
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,33.982877,43.879477,53.819722,58.770173,58.228965,67.238285,43.203166,40.925208,44.530412,54.248889
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.461360,40.166567,48.525258,45.211190,58.836387,66.252705,39.676606,37.621297,48.769067,51.394929


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.764337   49.035446   39.564176   
        2    0.082400       7.0    69.885701   91.828721   80.274245   
        3    0.154366       1.0   108.031336  142.393593  123.869045   
        4    0.067659       7.0   154.414716  190.951232  171.729119   
2       1    0.086461       2.0    37.614954   53.022743   44.637212   
        2    0.087900       5.0    77.331738   98.456745   87.292951   
        3    0.067659       5.0   126.928165  152.218743  139.218959   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.781299          3.677269         0.110735         0.0   
        2     80.402428          4.384634         0.069438         0.0   
        3    124.221190          4.819565         0.070663         0.0   
        4    171.999932          5.146018         0.054311         0.0   
2       1     44.776217          3.797895         0.086945         0.0   
        2     87.431130          4.468955         0.061601         0.0   
        3    139.291863          4.935492         0.046471         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         31.764337       49.035446   
        2                0.0         34.486960       47.505434   
        3                0.0         32.132384       58.328542   
        4                0.0         41.737356       54.536690   
2       1                0.0         37.614954       53.022743   
        2                0.0         35.832556       50.552995   
        3                0.0         45.385207       59.019893   

             personal_median  
team_id leg                   
1       1          39.564176  
        2          40.512037  
        3          43.358461  
        4          47.657695  
2       1          44.637212  
        2          42.488228  
        3          51.724696

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]
local_start_ts = pd.Timestamp(start_timestamp).tz_localize(None)

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= local_start_ts)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= local_start_ts)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9587   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.7963   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.111400       1.0    31.764337   49.035446   39.564176   
        2    0.082400       7.0    69.885701   91.828721   80.274245   
        3    0.154366       1.0   108.031336  142.393593  123.869045   
        4    0.067659       7.0   154.414716  190.951232  171.729119   
2       1    0.086461       2.0    37.614954   53.022743   44.637212   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.781299  ...         0.110735         0.0              0.0   
        2     80.402428  ...         0.069438         0.0              0.0   
        3    124.221190  ...         0.070663         0.0              0.0   
        4    171.999932  ...         0.054311         0.0              0.0   
2       1     44.776217  ...         0.086945         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           31.764337       49.035446        39.564176   
        2           34.486960       47.505434        40.512037   
        3           32.132384       58.328542        43.358461   
        4           41.737356       54.536690        47.657695   
2       1           37.614954       53.022743        44.637212   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:33.850540876 2019-06-15 14:31:45.860197783   
        2   2019-06-15 15:20:16.454704523 2019-06-15 15:09:53.142079830   
        3   2019-06-15 16:03:52.142708540 2019-06-15 15:48:01.880183935   
        4   2019-06-15 16:51:43.747113228 2019-06-15 16:34:24.882945538   
2       1   2019-06-15 14:44:38.232706785 2019-06-15 14:37:36.897221565   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:49:02.126787901                  17.271110  
        2   2019-06-15 15:31:49.723230362                  13.018473  
        3   2019-06-15 16:22:23.615605354                  26.196158  
        4   2019-06-15 17:10:57.073938608                  12.799334  
2       1   2019-06-15 14:53:01.364593744                  15.407790  

[5 rows x 21 columns]

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

for_print[for_print['team'].str.contains("Stadin")]
#for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,
1,1,Tilda Östberg,1.0,31.76,39.56,49.04,17.27
1,2,Magdalena Olsson,7.0,34.49,40.51,47.51,13.02
1,3,Julia Jakob,1.0,32.13,43.36,58.33,26.20
1,4,Tove Alexandersson,7.0,41.74,47.66,54.54,12.80
2,1,Amanda Hansson,2.0,37.61,44.64,53.02,15.41
...,...,...,...,...,...,...,...
1722,4,Johanna Liinamaa,3.0,72.72,89.59,110.31,37.59
1723,1,Elina Holstikko,0.0,62.95,89.86,129.91,66.96
1723,2,Mirka Leinonen,0.0,59.96,85.68,122.74,62.78


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1332,341,1,Rasti-Jyry 1,Jola Maisala,8.0,2.188600,0.126500,41.685819,53.477528,69.043674,3.979638,0.127310,2019-06-15 11:41:41.149122953+00:00,2019-06-15 12:09:02.620449543+00:00,2019-06-15 11:53:28.651698351+00:00,0.0000,0.0
1333,341,2,Rasti-Jyry 1,Kaija Rantoja,7.0,2.296600,0.144100,42.803399,56.764804,74.687016,4.706748,0.096995,2019-06-15 12:31:37.273753643+00:00,2019-06-15 13:13:44.646178961+00:00,2019-06-15 12:50:32.919028282+00:00,0.0000,0.0
1334,341,3,Rasti-Jyry 1,Jenni Castren,5.0,2.334200,0.141700,57.109744,75.251055,99.744614,5.229527,0.081773,2019-06-15 13:39:09.803307056+00:00,2019-06-15 14:39:20.067618608+00:00,2019-06-15 14:06:38.260004997+00:00,0.0000,0.0
1335,341,4,Rasti-Jyry 1,Inka Lemmetyinen,1.0,2.293200,0.168000,56.043655,78.086406,108.523993,5.582401,0.076726,2019-06-15 14:48:52.787197351+00:00,2019-06-15 16:08:10.994879246+00:00,2019-06-15 15:25:32.156561852+00:00,0.0000,0.0
2076,528,1,Rasti-Jyry 2,Päivi Taponen,3.0,2.361600,0.167728,45.761914,63.611141,88.791599,4.152578,0.168483,2019-06-15 11:45:45.714834213+00:00,2019-06-15 12:28:47.495910168+00:00,2019-06-15 12:03:36.668455601+00:00,0.0000,0.0
2077,528,2,Rasti-Jyry 2,Mervi Luurila,2.0,2.473200,0.183049,47.276389,67.728491,96.598825,4.884137,0.125284,2019-06-15 12:43:23.908500433+00:00,2019-06-15 13:49:52.620139122+00:00,2019-06-15 13:12:03.111095905+00:00,0.0000,0.0
2078,528,3,Rasti-Jyry 2,Suvi Tiainen,2.0,2.646600,0.187243,71.382054,103.464107,147.504423,5.467564,0.108685,2019-06-15 14:11:34.643913746+00:00,2019-06-15 15:53:11.823913336+00:00,2019-06-15 14:56:41.056494236+00:00,0.0000,0.0
2079,528,4,Rasti-Jyry 2,Tiina Isoniemi,6.0,2.153800,0.067659,59.781366,68.079742,77.745549,5.721621,0.085874,2019-06-15 15:18:48.432491541+00:00,2019-06-15 17:02:31.793556690+00:00,2019-06-15 16:05:12.496287823+00:00,0.1181,0.0
2916,740,1,Rasti-Jyry 4,Mila Viksilä,0.0,2.287041,0.182322,41.398642,58.886933,84.406306,4.076350,0.180610,2019-06-15 11:41:23.918532133+00:00,2019-06-15 12:24:24.378333092+00:00,2019-06-15 11:58:53.215980530+00:00,0.0000,0.0
2917,740,2,Rasti-Jyry 4,Tiina Sidensnöre,7.0,2.262600,0.146600,41.167379,54.883312,73.073048,4.740528,0.118175,2019-06-15 12:30:59.797559738+00:00,2019-06-15 13:25:16.361959934+00:00,2019-06-15 12:54:22.811836004+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(6728, 17)

In [20]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean", "count"]).round(2)


personal_median       personal_estimate_interval      
                    mean count                       mean count
num_runs                                                       
0.0                82.51  1646                      65.65  1646
1.0                80.72  1021                      61.84  1021
2.0                81.42   730                      58.03   730
3.0                78.09   697                      55.89   697
4.0                76.82   557                      32.72   557
5.0                75.57   566                      35.26   566
6.0                73.00   551                      33.59   551
7.0                71.35   571                      32.78   571
8.0                71.37   386                      32.30   386
9.0                84.15     2                      36.38     2
10.0               62.37     1                      16.69     1

In [21]:
for_web["interval_length"] = for_web.fin_time_end95 - for_web.fin_time_start95
for_web["interval_length"].median()
#for_web[['leg', 'interval_length', "num_runs"]].info()

Timedelta('0 days 01:08:56.045109')

In [22]:
for_web[['leg', 'interval_length', "num_runs"]].groupby('leg').agg(["mean", "count"]).round(2)

num_runs      
        mean count
leg               
1       2.97  1682
2       2.75  1682
3       2.98  1682
4       3.23  1682

In [ ]:
import datetime
datetime.datetime.now()